<a href="https://colab.research.google.com/github/julianencisoizquierdo/NLP_Python_Airline_Customer_Service/blob/main/NLP_BI_Airline_Industry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# suppress warnings
import warnings;
warnings.filterwarnings('ignore');

# common imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import html
import math
import re
import glob
import os
import sys
import json
import random
import pprint as pp
import textwrap
import sqlite3
import logging
from fractions import Fraction

import spacy
import nltk

import seaborn as sns
sns.set_style("darkgrid")

from tqdm.auto import tqdm
# register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
tqdm.pandas()


# Machine learning packages
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay, f1_score, cohen_kappa_score, roc_auc_score, average_precision_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder


# pandas display options
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html#available-options
pd.options.display.max_columns = 30 # default 20
pd.options.display.max_rows = 60 # default 60
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision = 2
pd.options.display.max_colwidth = 200 # default 50; -1 = all
# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# np.set_printoptions(edgeitems=3) # default 3

plot_params = {'figure.figsize': (8, 4),
               'axes.labelsize': 'large',
               'axes.titlesize': 'large',
               'xtick.labelsize': 'large',
               'ytick.labelsize':'large',
               'figure.dpi': 100}




###**Exploratory Data Analysis**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP Group Project/Datasets/airline.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1377 entries, 0 to 1376
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   weekday  1377 non-null   object
 1   month    1377 non-null   object
 2   date     1377 non-null   int64 
 3   year     1377 non-null   int64 
 4   text     1377 non-null   object
dtypes: int64(2), object(3)
memory usage: 53.9+ KB


In [ ]:
# Check for duplicated rows

duplicate_rows = df[df.duplicated()]

if not duplicate_rows.empty:
    print("There are duplicated rows in the dataset:")
    print(duplicate_rows)
else:
    print("There are no duplicated rows in the dataset.")

There are no duplicated rows in the dataset.


In [ ]:
# Print uniques

print(df['weekday'].unique())
print(df['year'].unique())
print(df['month'].unique())
print(df['date'].unique())

['Thu' 'Fri' 'Sat' 'Sun' 'Mon' 'Tue' 'Wed']
[2015]
['Oct']
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [ ]:
df.sample(10, random_state=11)

,weekday,month,date,year,text
22,Thu,Oct,1,2015,@iamdonovan Hi! Please DM your Delta ticket or confirmation number. I will check for the AZ confirmation number. *MR
513,Tue,Oct,6,2015,@realantmoney Sounds like we've let you down. Please let us know how we can help. *SB
220,Sat,Oct,3,2015,"@reeltake You can check-in to a flight 24hrs before, on line. *MR"
814,Fri,Oct,9,2015,@AllieRands ...that you please continue to check with the agents there at the airport for the most updated status on your flight. ... 3/4
782,Fri,Oct,9,2015,"@mattmogle There's no way to know what seat you would have, on which flight, if you miss your original connection Mr. Mogle. *NG"
349,Sun,Oct,4,2015,@LostConsultant Thanks for your patience all day. *PL
214,Sat,Oct,3,2015,@roxiozolins Are you in need of assistance? *DD
665,Thu,Oct,8,2015,@chaphya Our IT Team is aware of the website issues and are working diligently to get it resolved. My apologies for any inconvenience. *NG
1025,Mon,Oct,12,2015,@ChipL5 ...Please follow and DM your confirmation number. \n *RB 2/2
576,Wed,Oct,7,2015,@brimcleod That's what we like to hear! Tweet 'ya later. *EC


## **QUESTION 1 - What is the average length of a social customer service reply?**

In [ ]:
pd.options.display.max_rows = 30

df['username'] = df['text'].str.extract(r'@(\w+)')

countperusername_df = df['username'].value_counts().reset_index()
countperusername_df.rename(columns={'index': 'username', 'username': 'count_username'}, inplace = True)
countperusername_df

,username,count_username
0,nmsmith5,10
1,Real_J_Eddy,7
2,MelanieAlise87,6
3,jovenitti,6
4,Kyrrie_Twin,6
...,...,...
899,WinstonUK,1
900,p2peters,1
901,AirlinesDotOrg,1
902,abctraveler59,1


In [ ]:
# We add the count of how many times the same username shows up to the main dataset

df = df.merge(countperusername_df, how="left", on='username')
df

,weekday,month,date,year,text,username,count_username
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA,mjdout,1
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA",rmarkerm,1
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",checho85,1
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3",nealaa,3
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3
...,...,...,...,...,...,...,...
1372,Thu,Oct,15,2015,@satijp Woohoo! Way to go Marla and Mira! Happy Travels. :) *DD,satijp,1
1373,Thu,Oct,15,2015,@lukenbaugh1 You're welcome! Have a great day! *RD,lukenbaugh1,1
1374,Thu,Oct,15,2015,"@jeffcarp If you do not make your connection, the gate agent will advise of other options. My apology for the delay. *DD",jeffcarp,2
1375,Thu,Oct,15,2015,@jeffcarp ...719pm. *DD 2/2,jeffcarp,2


In [ ]:
pattern = r'(?<= )(\d/\d)'

df['iteration_id'] = df['text'].str.extract(pattern)
df

,weekday,month,date,year,text,username,count_username,iteration_id
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA,mjdout,1,NaN
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA",rmarkerm,1,NaN
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",checho85,1,NaN
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3",nealaa,3,3/3
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3
...,...,...,...,...,...,...,...,...
1372,Thu,Oct,15,2015,@satijp Woohoo! Way to go Marla and Mira! Happy Travels. :) *DD,satijp,1,NaN
1373,Thu,Oct,15,2015,@lukenbaugh1 You're welcome! Have a great day! *RD,lukenbaugh1,1,NaN
1374,Thu,Oct,15,2015,"@jeffcarp If you do not make your connection, the gate agent will advise of other options. My apology for the delay. *DD",jeffcarp,2,NaN
1375,Thu,Oct,15,2015,@jeffcarp ...719pm. *DD 2/2,jeffcarp,2,2/2


In [ ]:
# Check to make sure that we have only exctracted the desired patterns

print(df['iteration_id'].unique())

[nan '3/3' '2/3' '1/3' '2/2' '1/2' '4/4' '3/4' '2/4' '1/4']


In [ ]:
# The following messages are missing one the first part

m = (df['iteration_id'] == '2/2') & (df['count_username'] == 1)
df[m]

,weekday,month,date,year,text,username,count_username,iteration_id
296,Sun,Oct,4,2015,@amandalinfoot ...the airport customer service team. You would need to inquire directly with an agent. *RS 2/2,amandalinfoot,1,2/2
396,Mon,Oct,5,2015,@joshcollesano ...it at the airport. *WG 2/2,joshcollesano,1,2/2
996,Sun,Oct,11,2015,@nataliegow ...quickly and safely as possible. *SB 2/2,nataliegow,1,2/2
1176,Tue,Oct,13,2015,@scottswonger ...# so that I can review your reservation with you. My apologies. *JH 2/2,scottswonger,1,2/2
1356,Thu,Oct,15,2015,@mikeoliver93 ...Direct Message your Delta confirmation #. *JH 2/2,mikeoliver93,1,2/2


In [ ]:
df['weight'] = df.apply(lambda row: "1/" + str(int(row["iteration_id"].split("/")[-1])) if pd.notna(row['iteration_id']) else "1", axis=1)

df

,weekday,month,date,year,text,username,count_username,iteration_id,weight
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA,mjdout,1,NaN,1
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA",rmarkerm,1,NaN,1
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",checho85,1,NaN,1
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3",nealaa,3,3/3,1/3
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3,1/3
...,...,...,...,...,...,...,...,...,...
1372,Thu,Oct,15,2015,@satijp Woohoo! Way to go Marla and Mira! Happy Travels. :) *DD,satijp,1,NaN,1
1373,Thu,Oct,15,2015,@lukenbaugh1 You're welcome! Have a great day! *RD,lukenbaugh1,1,NaN,1
1374,Thu,Oct,15,2015,"@jeffcarp If you do not make your connection, the gate agent will advise of other options. My apology for the delay. *DD",jeffcarp,2,NaN,1
1375,Thu,Oct,15,2015,@jeffcarp ...719pm. *DD 2/2,jeffcarp,2,2/2,1/2


In [ ]:
import html

def text_cleaner(col):
    new_col = col.str.replace(r'https?://\S+', '', regex=True)
    new_col = new_col.str.replace(r'(@\w+)', '', regex=True)
    new_col = new_col.str.replace(r' (\d/\d)', '', regex=True)
    new_col = new_col.str.replace(r'\*.*$', '', regex=True)
    new_col = new_col.str.replace(r'\n', ' ', regex=True)
    new_col = new_col.str.replace(r'\.\.\.', '', regex=True)
    new_col = new_col.str.replace(r'([A-Z]{2})$', '', regex=True)
    new_col = new_col.apply(html.unescape)
    new_col = new_col.str.replace(r'&;', '&', regex=True)
    return new_col

df['clean_text'] = text_cleaner(df['text'])

df.tail(10)

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text
1367,Thu,Oct,15,2015,@Sennz Oh no! I will be happy to take a look. Please follow/DM your confirm #. *CM,Sennz,1,NaN,1,Oh no! I will be happy to take a look. Please follow/DM your confirm #.
1368,Thu,Oct,15,2015,@jenn_peterson Sorry for the delay. Hope you are on your way soon. *DD,jenn_peterson,1,NaN,1,Sorry for the delay. Hope you are on your way soon.
1369,Thu,Oct,15,2015,"@brendafay27 Yay, thanks for the shout out! Please follow and direct message me your conf# so I can forward your compliment. *AD",brendafay27,2,NaN,1,"Yay, thanks for the shout out! Please follow and direct message me your conf# so I can forward your compliment."
1370,Thu,Oct,15,2015,"@hayhunhad Thank you for the information. I will definitely forward your comments to the Operations team, Rick. *CM",hayhunhad,1,NaN,1,"Thank you for the information. I will definitely forward your comments to the Operations team, Rick."
1371,Thu,Oct,15,2015,@mmmeincke Hi there. My apologies for the delay. I see the equipment is arriving late to DTW for DL5436. *CM,mmmeincke,1,NaN,1,Hi there. My apologies for the delay. I see the equipment is arriving late to DTW for DL5436.
1372,Thu,Oct,15,2015,@satijp Woohoo! Way to go Marla and Mira! Happy Travels. :) *DD,satijp,1,NaN,1,Woohoo! Way to go Marla and Mira! Happy Travels. :)
1373,Thu,Oct,15,2015,@lukenbaugh1 You're welcome! Have a great day! *RD,lukenbaugh1,1,NaN,1,You're welcome! Have a great day!
1374,Thu,Oct,15,2015,"@jeffcarp If you do not make your connection, the gate agent will advise of other options. My apology for the delay. *DD",jeffcarp,2,NaN,1,"If you do not make your connection, the gate agent will advise of other options. My apology for the delay."
1375,Thu,Oct,15,2015,@jeffcarp ...719pm. *DD 2/2,jeffcarp,2,2/2,1/2,719pm.
1376,Thu,Oct,15,2015,@svchappel That sounds yummy. :) *CM,svchappel,4,NaN,1,That sounds yummy. :)


In [ ]:
df["length_char"] = df["clean_text"].apply(lambda x : len(x))

df['length_words'] = df['clean_text'].apply(lambda x: len(str(x).split()))

df.tail(10)

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_char,length_words
1367,Thu,Oct,15,2015,@Sennz Oh no! I will be happy to take a look. Please follow/DM your confirm #. *CM,Sennz,1,NaN,1,Oh no! I will be happy to take a look. Please follow/DM your confirm #.,73,15
1368,Thu,Oct,15,2015,@jenn_peterson Sorry for the delay. Hope you are on your way soon. *DD,jenn_peterson,1,NaN,1,Sorry for the delay. Hope you are on your way soon.,53,11
1369,Thu,Oct,15,2015,"@brendafay27 Yay, thanks for the shout out! Please follow and direct message me your conf# so I can forward your compliment. *AD",brendafay27,2,NaN,1,"Yay, thanks for the shout out! Please follow and direct message me your conf# so I can forward your compliment.",113,20
1370,Thu,Oct,15,2015,"@hayhunhad Thank you for the information. I will definitely forward your comments to the Operations team, Rick. *CM",hayhunhad,1,NaN,1,"Thank you for the information. I will definitely forward your comments to the Operations team, Rick.",102,16
1371,Thu,Oct,15,2015,@mmmeincke Hi there. My apologies for the delay. I see the equipment is arriving late to DTW for DL5436. *CM,mmmeincke,1,NaN,1,Hi there. My apologies for the delay. I see the equipment is arriving late to DTW for DL5436.,95,18
1372,Thu,Oct,15,2015,@satijp Woohoo! Way to go Marla and Mira! Happy Travels. :) *DD,satijp,1,NaN,1,Woohoo! Way to go Marla and Mira! Happy Travels. :),53,10
1373,Thu,Oct,15,2015,@lukenbaugh1 You're welcome! Have a great day! *RD,lukenbaugh1,1,NaN,1,You're welcome! Have a great day!,35,6
1374,Thu,Oct,15,2015,"@jeffcarp If you do not make your connection, the gate agent will advise of other options. My apology for the delay. *DD",jeffcarp,2,NaN,1,"If you do not make your connection, the gate agent will advise of other options. My apology for the delay.",109,20
1375,Thu,Oct,15,2015,@jeffcarp ...719pm. *DD 2/2,jeffcarp,2,2/2,1/2,719pm.,8,1
1376,Thu,Oct,15,2015,@svchappel That sounds yummy. :) *CM,svchappel,4,NaN,1,That sounds yummy. :),23,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1377 entries, 0 to 1376
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   weekday         1377 non-null   object
 1   month           1377 non-null   object
 2   date            1377 non-null   int64 
 3   year            1377 non-null   int64 
 4   text            1377 non-null   object
 5   username        1377 non-null   object
 6   count_username  1377 non-null   int64 
 7   iteration_id    347 non-null    object
 8   weight          1377 non-null   object
 9   clean_text      1377 non-null   object
 10  length_char     1377 non-null   int64 
 11  length_words    1377 non-null   int64 
dtypes: int64(5), object(7)
memory usage: 139.9+ KB


In [ ]:
df['weight_float'] = df['weight'].apply(lambda x: float(Fraction(x)))

df

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_char,length_words,weight_float
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA,mjdout,1,NaN,1,I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience.,109,18,1.00
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA",rmarkerm,1,NaN,1,"Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know.",98,18,1.00
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",checho85,1,NaN,1,"I can check, pls follow and DM your confirmation # for review.",66,12,1.00
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3",nealaa,3,3/3,1/3,"Alerts, pls check here:",26,4,0.33
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3,1/3,"advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &",116,20,0.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,Thu,Oct,15,2015,@satijp Woohoo! Way to go Marla and Mira! Happy Travels. :) *DD,satijp,1,NaN,1,Woohoo! Way to go Marla and Mira! Happy Travels. :),53,10,1.00
1373,Thu,Oct,15,2015,@lukenbaugh1 You're welcome! Have a great day! *RD,lukenbaugh1,1,NaN,1,You're welcome! Have a great day!,35,6,1.00
1374,Thu,Oct,15,2015,"@jeffcarp If you do not make your connection, the gate agent will advise of other options. My apology for the delay. *DD",jeffcarp,2,NaN,1,"If you do not make your connection, the gate agent will advise of other options. My apology for the delay.",109,20,1.00
1375,Thu,Oct,15,2015,@jeffcarp ...719pm. *DD 2/2,jeffcarp,2,2/2,1/2,719pm.,8,1,0.50


In [ ]:
df['final_length_char'] = df.apply(lambda row: (row['weight_float']) * row['length_char'], axis=1)

df['final_length_words'] = df.apply(lambda row: (row['weight_float']) * row['length_words'], axis=1)

df.head(10)

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_char,length_words,weight_float,final_length_char,final_length_words
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA,mjdout,1,NaN,1,I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience.,109,18,1.00,109.00,18.00
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA",rmarkerm,1,NaN,1,"Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know.",98,18,1.00,98.00,18.00
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",checho85,1,NaN,1,"I can check, pls follow and DM your confirmation # for review.",66,12,1.00,66.00,12.00
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3",nealaa,3,3/3,1/3,"Alerts, pls check here:",26,4,0.33,8.67,1.33
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3,1/3,"advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &",116,20,0.33,38.67,6.67
5,Thu,Oct,1,2015,@nealaa Hi. Our meteorologist team is aware of Hurricane Joaquin &amp; monitors weather conditions at all times. At this time a weather... 1/3,nealaa,3,1/3,1/3,Hi. Our meteorologist team is aware of Hurricane Joaquin & monitors weather conditions at all times. At this time a weather,124,21,0.33,41.33,7.00
6,Thu,Oct,1,2015,@BigGucciQueen This is your direct dial number + 43 (0)1 360 277 3461. *DD,BigGucciQueen,2,NaN,1,This is your direct dial number + 43 (0)1 360 277 3461.,58,12,1.00,58.00,12.00
7,Thu,Oct,1,2015,@marxypoo ...for any inconvenience. *JH 3/3,marxypoo,3,3/3,1/3,for any inconvenience.,25,3,0.33,8.33,1.00
8,Thu,Oct,1,2015,@marxypoo ...the system &amp; when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies... 2/3,marxypoo,3,2/3,1/3,the system & when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies,117,22,0.33,39.00,7.33
9,Thu,Oct,1,2015,"@marxypoo Hi Marx. This is a known issue &amp; our IT team members are working on a fix. Be assured, once you make the change it is in... 1/3",marxypoo,3,1/3,1/3,"Hi Marx. This is a known issue & our IT team members are working on a fix. Be assured, once you make the change it is in",121,27,0.33,40.33,9.00


In [ ]:
avg_len_char = df['final_length_char'].mean()

avg_len_words = df['final_length_words'].mean()

print(f'The average length of a social customer service reply is {avg_len_char:.2f} characters')
print(f'The average length of a social customer service reply is {avg_len_words:.2f} words')

The average length of a social customer service reply is 63.60 characters
The average length of a social customer service reply is 11.22 words


##**QUESTION 2 - What types of links were referenced most often?**

In [ ]:
df_q2 = df[['text']]

url_pattern = r'https?://\S+'
df_q2['url_count'] = df_q2['text'].str.count(url_pattern)

# Check where url_count == 0 to see if we are missing some
fil = (df_q2['url_count'] == 0)
df_q2[fil].sample(n=10)

,text,url_count
1212,"@jbossie42 Yes, it looks like it'll be better for your friend to keep speaking with Cargo concerning this. *WG",0
960,@milos002 You are most welcome! *SB,0
814,@AllieRands ...that you please continue to check with the agents there at the airport for the most updated status on your flight. ... 3/4,0
454,@4162_514 You're ticketed for the 7:30P (DL1638). DL1738 is the 6:20P you want to SDC/standby for? DL1738 is departing from gate T3.... 1/2,0
752,"@penaetis Oh Ok. I understand. Paxs traveling on Delta Connection flts, &amp; only those flts with 50 seats or less, are only permitted... 1/3",0
1185,@smartysarahs I agree! They are the best! Thanks for the compliment. *AN,0
190,@HarrisIII ...experience today. *NG 3/3,0
1367,@Sennz Oh no! I will be happy to take a look. Please follow/DM your confirm #. *CM,0
666,@joymh You're welcome! *RD,0
380,@benjablake ...please let me know. You may DM details. *RS 2/2,0


In [ ]:
phone_pat = r'(\(?\d{3}\)?[\s-]?\d{3}[\s-]?\d{4})'
df_q2['phone_count'] = df_q2['text'].str.count(phone_pat)

# Check where phone_count == 0 to see if we are missing some
mk = (df_q2['phone_count'] == 0)
df_q2[mk].sample(n=7)

,text,url_count,phone_count
386,"@dcbergeson Thank you. Could you follow this account, so I can DM you? *WG",0,0
1339,@_GJR_ Sorry for the delay. Pls follow/DM your confirm # for your privacy. *RD,0,0
1359,@svchappel It's my pleasure. :) *CM,0,0
436,@CCH211 Yea! So happy the app is working for you now. *PL,0,0
122,@LifeAsPosh Hi. It depends on the rules of your ticket. Please follow/DM your ticket # and I'll be happy to check on this for you. *AB,0,0
780,@CWunder Your flight is operated by Air Europa. You will check in at the Airport with UX. *DD,0,0
482,"@gopherfan2 ...navigational assistance is needed to complete the upgrade online, pls call our online customer support team at... 2/3",0,0


In [ ]:
DM_pat = "(DM)"
df_q2['DM'] = df_q2['text'].str.count(DM_pat)

# Check where DM == 0 to see if we are missing some
mk3 = (df_q2['DM'] == 1)
df_q2[mk3]

,text,url_count,phone_count,DM
2,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",0,0,1
21,@papapaintvids Sorry to hear. Pls follow/DM your confirmation # and I'll take a look to see what's on going on. *NG,0,0,1
22,@iamdonovan Hi! Please DM your Delta ticket or confirmation number. I will check for the AZ confirmation number. *MR,0,0,1
25,@Sam_L_Shead So sorry to hear Sam. Pls follow/DM your confirmation # if rebooking assistance is needed. *NG,0,0,1
43,@zdlatham Sorry about that! Please follow/DM your confirm #. *KM,0,0,1
...,...,...,...,...
1336,@d_wave27 Please follow and DM I have found the terms but its a bit lengthy. *KC,0,0,1
1338,@crodrigu38 I see. Can you DM your confirm #? I'd like to take a closer look. *KM,0,0,1
1339,@_GJR_ Sorry for the delay. Pls follow/DM your confirm # for your privacy. *RD,0,0,1
1355,@IMgabimagalhaes Yikes! That's not the response we we're going for! Can you follow/DM your flt # Gabriela? *KM,0,0,1


In [ ]:
DM_pat_2 = "(direct message)\s"

df_q2['DM_2'] = df_q2['text'].str.count(DM_pat_2, flags=re.IGNORECASE)
df_q2.sample(n=7)

,text,url_count,phone_count,DM,DM_2
1351,@giseleparc Give me a moment let me check for you. *HW,0,0,0,0
1262,@steveuk10 ...30days for a response. Have you tried calling? 1-800-455-2720 option 2/4. *EC 2/2,0,1,0,0
915,@ricksouthend Hmm....I can definitely review your case &amp; forward any issues to their leadership team. Did you get a case number? *VM,0,0,0,0
963,"@laurawolfmother Hey Laura, sorry you're not enjoying your journey with us. If I can be of assistance, pls let me know. *VM",0,0,0,0
572,"@LittlePasch You're most welcome! Timing will only be a deciding factor against another Gold Medallion with same status, class of... 1/2",0,0,0,0
17,@gim_esther Sorry to hear Esther. I will cancel your flights for you. *NG,0,0,0,0
550,"@himolin Hi. My apologies for the missing bag. For your privacy, pls follow &amp; Direct Message your Delta Lost Baggage Claim Report # &amp;... 1/2",0,0,0,1


In [ ]:
pd.set_option('display.max_rows', 10)

df_q2['DM_count'] = df_q2['DM'] + df_q2['DM_2']
df_q2.drop(columns=['DM', 'DM_2'], inplace = True)
df_q2.sample(n=7)

,text,url_count,phone_count,DM_count
1135,@bryanzulker Gate passes are only provided at the counter by the ticket counter agent. I apologize for the inconvenience. *ML,0,0,0
291,@MClark_Live WooHoo! We appreciate your loyalty Mark:) Please read this link details regarding your Medallion status: ... 1/2,0,0,0
97,@airrayd Wonderful to hear. Thanks so much for sharing and for choosing Delta. *AA,0,0,0
1363,"@morettiphd Hi. For your privacy, pls follow &amp; Direct Message your Delta confirmation # for review. *JH",0,0,1
1190,"@wjoooooood If you have not received a confirmation number with a ticket number issued in your name, there will be no charge and any... 1/2",0,0,0
141,@_jerk_sauce I am unable to see your email. Please follow/DM your details. *PL,0,0,1
778,@merlin_aka_sly My apologies for the delay in receiving your miles. Currently the desk which handles these request is closed. Pls... 1/2,0,0,0


In [ ]:
url_tot = df_q2['url_count'].sum()
phone_tot = df_q2['phone_count'].sum()
DM_tot = df_q2['DM_count'].sum()

print(f'Total URL references: {url_tot}')
print(f'Total phone number references: {phone_tot}')
print(f'Total direct messages references: {DM_tot}')

Total URL references: 59
Total phone number references: 77
Total direct messages references: 201


## **QUESTION 3 - How many people should be on a social media customer service team?**

In [ ]:
df_q3 = df.drop(columns=['final_length_char', 'length_char'])

df_q3['worker_id'] = df_q3['text'].str.extract(r'\*(\w+)')
df_q3

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA,mjdout,1,NaN,1,I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience.,18,1.00,18.00,AA
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA",rmarkerm,1,NaN,1,"Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know.",18,1.00,18.00,AA
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",checho85,1,NaN,1,"I can check, pls follow and DM your confirmation # for review.",12,1.00,12.00,AA
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3",nealaa,3,3/3,1/3,"Alerts, pls check here:",4,0.33,1.33,JH
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3,1/3,"advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &",20,0.33,6.67,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,Thu,Oct,15,2015,@satijp Woohoo! Way to go Marla and Mira! Happy Travels. :) *DD,satijp,1,NaN,1,Woohoo! Way to go Marla and Mira! Happy Travels. :),10,1.00,10.00,DD
1373,Thu,Oct,15,2015,@lukenbaugh1 You're welcome! Have a great day! *RD,lukenbaugh1,1,NaN,1,You're welcome! Have a great day!,6,1.00,6.00,RD
1374,Thu,Oct,15,2015,"@jeffcarp If you do not make your connection, the gate agent will advise of other options. My apology for the delay. *DD",jeffcarp,2,NaN,1,"If you do not make your connection, the gate agent will advise of other options. My apology for the delay.",20,1.00,20.00,DD
1375,Thu,Oct,15,2015,@jeffcarp ...719pm. *DD 2/2,jeffcarp,2,2/2,1/2,719pm.,1,0.50,0.50,DD


In [ ]:
msk1 = df_q3['worker_id'].notna()
df_worker_notna = df_q3[msk1]

In [ ]:
msk2 = df_q3['worker_id'].isna()
df_worker_na = df_q3[msk2]
df_worker_na

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3,1/3,"advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &",20,0.33,6.67,NaN
5,Thu,Oct,1,2015,@nealaa Hi. Our meteorologist team is aware of Hurricane Joaquin &amp; monitors weather conditions at all times. At this time a weather... 1/3,nealaa,3,1/3,1/3,Hi. Our meteorologist team is aware of Hurricane Joaquin & monitors weather conditions at all times. At this time a weather,21,0.33,7.00,NaN
8,Thu,Oct,1,2015,@marxypoo ...the system &amp; when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies... 2/3,marxypoo,3,2/3,1/3,the system & when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies,22,0.33,7.33,NaN
9,Thu,Oct,1,2015,"@marxypoo Hi Marx. This is a known issue &amp; our IT team members are working on a fix. Be assured, once you make the change it is in... 1/3",marxypoo,3,1/3,1/3,"Hi Marx. This is a known issue & our IT team members are working on a fix. Be assured, once you make the change it is in",27,0.33,9.00,NaN
14,Thu,Oct,1,2015,@bar_right Hi Pete. That is not good to hear. For questions or comments regarding the wifi service pls tweet @Gogo. A team member... 1/2,bar_right,2,1/2,1/2,Hi Pete. That is not good to hear. For questions or comments regarding the wifi service pls tweet . A team member,22,0.50,11.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,Thu,Oct,15,2015,"@terrytravels1 To ensure we're able to meet our customer demand, we have a global presence in providing phone support within the U.S.... 1/2",terrytravels1,2,1/2,1/2,"To ensure we're able to meet our customer demand, we have a global presence in providing phone support within the U.S.",21,0.50,10.50,NaN
1311,Thu,Oct,15,2015,"@heleneheld Currently, our Reservations Team are not accepting calls due to a system maintenance. Are you only wanting to confirm... 1/2",heleneheld,3,1/2,1/2,"Currently, our Reservations Team are not accepting calls due to a system maintenance. Are you only wanting to confirm",19,0.50,9.50,NaN
1313,Thu,Oct,15,2015,@ElGatoBravo Currently they are not accepting calls due to a system maintenance. Please try back momentarily. I apologize for any... 1/2,ElGatoBravo,2,1/2,1/2,Currently they are not accepting calls due to a system maintenance. Please try back momentarily. I apologize for any,19,0.50,9.50,NaN
1348,Thu,Oct,15,2015,"@KerKilbourne ...security screening locations, contact TSA), 773-601-1817 (Airport Transit System), 773-686-7532 (Parking... 2/3",KerKilbourne,3,2/3,1/3,"security screening locations, contact TSA), 773-601-1817 (Airport Transit System), 773-686-7532 (Parking",11,0.33,3.67,NaN


In [ ]:
pattern = r'([A-Z]{2})$'

df_worker_na['worker_id'] = df_worker_na['text'].str.extract(pattern)

# Check
filtered_df = df_worker_na[df_worker_na['worker_id'].notna()]
filtered_df.head(n=11)

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
517,Tue,Oct,6,2015,@okg8tr My pleasure! :-) AN,okg8tr,4,NaN,1,My pleasure! :-),3,1.00,3.00,AN
518,Tue,Oct,6,2015,"@sootawn Hi, it could take up to 7 to 10 business days after filling out the form. VI",sootawn,1,NaN,1,"Hi, it could take up to 7 to 10 business days after filling out the form.",16,1.00,16.00,VI
521,Tue,Oct,6,2015,@okg8tr Hi Randy. Let me look into this for you AN,okg8tr,4,NaN,1,Hi Randy. Let me look into this for you,9,1.00,9.00,AN
522,Tue,Oct,6,2015,"@zeekcrse Hi, I'll be happy to check the availability of upgrades on your flight. Pls follow/DM your conf#. AD",zeekcrse,1,NaN,1,"Hi, I'll be happy to check the availability of upgrades on your flight. Pls follow/DM your conf#.",17,1.00,17.00,AD
523,Tue,Oct,6,2015,@harrelson_b I am so sorry that you feel that way. This is not a generic response. Sorry there was no available discounts for your trip. HW,harrelson_b,1,NaN,1,I am so sorry that you feel that way. This is not a generic response. Sorry there was no available discounts for your trip.,24,1.00,24.00,HW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Tue,Oct,6,2015,@kate_wiley1 Welcome aboard! Enjoy! ML,kate_wiley1,1,NaN,1,Welcome aboard! Enjoy!,3,1.00,3.00,ML
531,Tue,Oct,6,2015,@MarneyReid I love it! Keep the momentum going! :-) AN,MarneyReid,1,NaN,1,I love it! Keep the momentum going! :-),8,1.00,8.00,AN
533,Tue,Oct,6,2015,"@jperrin83 Oh, no! Let me see if I can help. Pls follow and DM the file reference number. TP",jperrin83,2,NaN,1,"Oh, no! Let me see if I can help. Pls follow and DM the file reference number.",17,1.00,17.00,TP
534,Tue,Oct,6,2015,"@SA_Thompson Goooood morning Steven, welcome home, Sir! We are happy that you're flying with us again! Thanks for your loyalty to Delta! CK",SA_Thompson,1,NaN,1,"Goooood morning Steven, welcome home, Sir! We are happy that you're flying with us again! Thanks for your loyalty to Delta!",21,1.00,21.00,CK


In [ ]:
df_2 = pd.concat([df_worker_notna, df_worker_na], axis=0)
df_2

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA,mjdout,1,NaN,1,I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience.,18,1.00,18.00,AA
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA",rmarkerm,1,NaN,1,"Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know.",18,1.00,18.00,AA
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",checho85,1,NaN,1,"I can check, pls follow and DM your confirmation # for review.",12,1.00,12.00,AA
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3",nealaa,3,3/3,1/3,"Alerts, pls check here:",4,0.33,1.33,JH
6,Thu,Oct,1,2015,@BigGucciQueen This is your direct dial number + 43 (0)1 360 277 3461. *DD,BigGucciQueen,2,NaN,1,This is your direct dial number + 43 (0)1 360 277 3461.,12,1.00,12.00,DD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,Thu,Oct,15,2015,"@terrytravels1 To ensure we're able to meet our customer demand, we have a global presence in providing phone support within the U.S.... 1/2",terrytravels1,2,1/2,1/2,"To ensure we're able to meet our customer demand, we have a global presence in providing phone support within the U.S.",21,0.50,10.50,NaN
1311,Thu,Oct,15,2015,"@heleneheld Currently, our Reservations Team are not accepting calls due to a system maintenance. Are you only wanting to confirm... 1/2",heleneheld,3,1/2,1/2,"Currently, our Reservations Team are not accepting calls due to a system maintenance. Are you only wanting to confirm",19,0.50,9.50,NaN
1313,Thu,Oct,15,2015,@ElGatoBravo Currently they are not accepting calls due to a system maintenance. Please try back momentarily. I apologize for any... 1/2,ElGatoBravo,2,1/2,1/2,Currently they are not accepting calls due to a system maintenance. Please try back momentarily. I apologize for any,19,0.50,9.50,NaN
1348,Thu,Oct,15,2015,"@KerKilbourne ...security screening locations, contact TSA), 773-601-1817 (Airport Transit System), 773-686-7532 (Parking... 2/3",KerKilbourne,3,2/3,1/3,"security screening locations, contact TSA), 773-601-1817 (Airport Transit System), 773-686-7532 (Parking",11,0.33,3.67,NaN


In [ ]:
msk5 = df_2['worker_id'].notna()
df_w_notna = df_2[msk5]
df_w_notna

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
0,Thu,Oct,1,2015,@mjdout I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience. *AA,mjdout,1,NaN,1,I know that can be frustrating..we hope to have you parked and deplaned shortly. Thanks for your patience.,18,1.00,18.00,AA
1,Thu,Oct,1,2015,"@rmarkerm Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know. *AA",rmarkerm,1,NaN,1,"Terribly sorry for the inconvenience. If we can be of assistance at this time, pls let us know.",18,1.00,18.00,AA
2,Thu,Oct,1,2015,"@checho85 I can check, pls follow and DM your confirmation # for review. *AA",checho85,1,NaN,1,"I can check, pls follow and DM your confirmation # for review.",12,1.00,12.00,AA
3,Thu,Oct,1,2015,"@nealaa ...Alerts, pls check here: http://t.co/0jlcZnT95Q *JH 3/3",nealaa,3,3/3,1/3,"Alerts, pls check here:",4,0.33,1.33,JH
6,Thu,Oct,1,2015,@BigGucciQueen This is your direct dial number + 43 (0)1 360 277 3461. *DD,BigGucciQueen,2,NaN,1,This is your direct dial number + 43 (0)1 360 277 3461.,12,1.00,12.00,DD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Tue,Oct,6,2015,@kate_wiley1 Welcome aboard! Enjoy! ML,kate_wiley1,1,NaN,1,Welcome aboard! Enjoy!,3,1.00,3.00,ML
531,Tue,Oct,6,2015,@MarneyReid I love it! Keep the momentum going! :-) AN,MarneyReid,1,NaN,1,I love it! Keep the momentum going! :-),8,1.00,8.00,AN
533,Tue,Oct,6,2015,"@jperrin83 Oh, no! Let me see if I can help. Pls follow and DM the file reference number. TP",jperrin83,2,NaN,1,"Oh, no! Let me see if I can help. Pls follow and DM the file reference number.",17,1.00,17.00,TP
534,Tue,Oct,6,2015,"@SA_Thompson Goooood morning Steven, welcome home, Sir! We are happy that you're flying with us again! Thanks for your loyalty to Delta! CK",SA_Thompson,1,NaN,1,"Goooood morning Steven, welcome home, Sir! We are happy that you're flying with us again! Thanks for your loyalty to Delta!",21,1.00,21.00,CK


In [ ]:
msk6 = df_2['worker_id'].isna()
df_w_na = df_2[msk6]
df_w_na

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
4,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3,1/3,"advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &",20,0.33,6.67,NaN
5,Thu,Oct,1,2015,@nealaa Hi. Our meteorologist team is aware of Hurricane Joaquin &amp; monitors weather conditions at all times. At this time a weather... 1/3,nealaa,3,1/3,1/3,Hi. Our meteorologist team is aware of Hurricane Joaquin & monitors weather conditions at all times. At this time a weather,21,0.33,7.00,NaN
8,Thu,Oct,1,2015,@marxypoo ...the system &amp; when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies... 2/3,marxypoo,3,2/3,1/3,the system & when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies,22,0.33,7.33,NaN
9,Thu,Oct,1,2015,"@marxypoo Hi Marx. This is a known issue &amp; our IT team members are working on a fix. Be assured, once you make the change it is in... 1/3",marxypoo,3,1/3,1/3,"Hi Marx. This is a known issue & our IT team members are working on a fix. Be assured, once you make the change it is in",27,0.33,9.00,NaN
14,Thu,Oct,1,2015,@bar_right Hi Pete. That is not good to hear. For questions or comments regarding the wifi service pls tweet @Gogo. A team member... 1/2,bar_right,2,1/2,1/2,Hi Pete. That is not good to hear. For questions or comments regarding the wifi service pls tweet . A team member,22,0.50,11.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,Thu,Oct,15,2015,"@terrytravels1 To ensure we're able to meet our customer demand, we have a global presence in providing phone support within the U.S.... 1/2",terrytravels1,2,1/2,1/2,"To ensure we're able to meet our customer demand, we have a global presence in providing phone support within the U.S.",21,0.50,10.50,NaN
1311,Thu,Oct,15,2015,"@heleneheld Currently, our Reservations Team are not accepting calls due to a system maintenance. Are you only wanting to confirm... 1/2",heleneheld,3,1/2,1/2,"Currently, our Reservations Team are not accepting calls due to a system maintenance. Are you only wanting to confirm",19,0.50,9.50,NaN
1313,Thu,Oct,15,2015,@ElGatoBravo Currently they are not accepting calls due to a system maintenance. Please try back momentarily. I apologize for any... 1/2,ElGatoBravo,2,1/2,1/2,Currently they are not accepting calls due to a system maintenance. Please try back momentarily. I apologize for any,19,0.50,9.50,NaN
1348,Thu,Oct,15,2015,"@KerKilbourne ...security screening locations, contact TSA), 773-601-1817 (Airport Transit System), 773-686-7532 (Parking... 2/3",KerKilbourne,3,2/3,1/3,"security screening locations, contact TSA), 773-601-1817 (Airport Transit System), 773-686-7532 (Parking",11,0.33,3.67,NaN


In [ ]:
mapping_and_count = df_w_notna.groupby(by=["date", 'username', "weight", 'worker_id']).size().reset_index(name="count")

mapping = mapping_and_count.drop(columns=["count"])
mapping

,date,username,weight,worker_id
0,1,Artistsupporter,1,WG
1,1,BigGucciQueen,1,DD
2,1,D0llabillwill,1/3,JH
3,1,DaneLopes,1,DD
4,1,Dirtyone23,1,NG
...,...,...,...,...
960,15,socialepicurean,1,CM
961,15,svchappel,1,CM
962,15,terrytravels1,1/2,AA
963,15,vicorly,1,ML


In [ ]:
df_w_na.drop(columns=["worker_id"], inplace = True)

df_w_na_mapped = pd.merge(df_w_na, mapping, how="left", on=["date", "weight", "username"])
df_w_na_mapped

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
0,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3,1/3,"advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &",20,0.33,6.67,JH
1,Thu,Oct,1,2015,@nealaa Hi. Our meteorologist team is aware of Hurricane Joaquin &amp; monitors weather conditions at all times. At this time a weather... 1/3,nealaa,3,1/3,1/3,Hi. Our meteorologist team is aware of Hurricane Joaquin & monitors weather conditions at all times. At this time a weather,21,0.33,7.00,JH
2,Thu,Oct,1,2015,@marxypoo ...the system &amp; when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies... 2/3,marxypoo,3,2/3,1/3,the system & when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies,22,0.33,7.33,JH
3,Thu,Oct,1,2015,"@marxypoo Hi Marx. This is a known issue &amp; our IT team members are working on a fix. Be assured, once you make the change it is in... 1/3",marxypoo,3,1/3,1/3,"Hi Marx. This is a known issue & our IT team members are working on a fix. Be assured, once you make the change it is in",27,0.33,9.00,JH
4,Thu,Oct,1,2015,@bar_right Hi Pete. That is not good to hear. For questions or comments regarding the wifi service pls tweet @Gogo. A team member... 1/2,bar_right,2,1/2,1/2,Hi Pete. That is not good to hear. For questions or comments regarding the wifi service pls tweet . A team member,22,0.50,11.00,JH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,Thu,Oct,15,2015,"@terrytravels1 To ensure we're able to meet our customer demand, we have a global presence in providing phone support within the U.S.... 1/2",terrytravels1,2,1/2,1/2,"To ensure we're able to meet our customer demand, we have a global presence in providing phone support within the U.S.",21,0.50,10.50,AA
182,Thu,Oct,15,2015,"@heleneheld Currently, our Reservations Team are not accepting calls due to a system maintenance. Are you only wanting to confirm... 1/2",heleneheld,3,1/2,1/2,"Currently, our Reservations Team are not accepting calls due to a system maintenance. Are you only wanting to confirm",19,0.50,9.50,AN
183,Thu,Oct,15,2015,@ElGatoBravo Currently they are not accepting calls due to a system maintenance. Please try back momentarily. I apologize for any... 1/2,ElGatoBravo,2,1/2,1/2,Currently they are not accepting calls due to a system maintenance. Please try back momentarily. I apologize for any,19,0.50,9.50,AN
184,Thu,Oct,15,2015,"@KerKilbourne ...security screening locations, contact TSA), 773-601-1817 (Airport Transit System), 773-686-7532 (Parking... 2/3",KerKilbourne,3,2/3,1/3,"security screening locations, contact TSA), 773-601-1817 (Airport Transit System), 773-686-7532 (Parking",11,0.33,3.67,JH


In [ ]:
fil = df_w_na_mapped["worker_id"].isna()
df_w_na_mapped[fil]

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
72,Tue,Oct,6,2015,@okg8tr ...http://t.co/AKUEAIxrX3 AN 2/2,okg8tr,4,2/2,1/2,,0,0.50,0.00,NaN
73,Tue,Oct,6,2015,"@okg8tr Got it. Wi-Fi service is available, but you must go through our partner GoGo . Please see the following link:... 1/2",okg8tr,4,1/2,1/2,"Got it. Wi-Fi service is available, but you must go through our partner GoGo . Please see the following link:",20,0.50,10.00,NaN
74,Tue,Oct,6,2015,@AnnCamden ...them. TP 2/2,AnnCamden,2,2/2,1/2,them.,1,0.50,0.50,NaN
75,Tue,Oct,6,2015,"@AnnCamden Oh, my! All the best to the passenger. Our staff is well-trained to handle emergencies...we care and will take care of... 1/2",AnnCamden,2,1/2,1/2,"Oh, my! All the best to the passenger. Our staff is well-trained to handle emergencieswe care and will take care of",21,0.50,10.50,NaN
174,Wed,Oct,14,2015,@steveuk10 We haven't forgotten about you. The response has been a bit longer to reply back in a timely fashion. It can take up... 1/2,steveuk10,2,1/2,1/2,We haven't forgotten about you. The response has been a bit longer to reply back in a timely fashion. It can take up,23,0.50,11.50,NaN


In [ ]:
df_3 = pd.concat([df_w_na_mapped, df_w_notna])
df_3

,weekday,month,date,year,text,username,count_username,iteration_id,weight,clean_text,length_words,weight_float,final_length_words,worker_id
0,Thu,Oct,1,2015,"@nealaa ...advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &amp;... 2/3",nealaa,3,2/3,1/3,"advisory has only been issued for the Bahamas, but that could change. To check for updates on Weather advisories &",20,0.33,6.67,JH
1,Thu,Oct,1,2015,@nealaa Hi. Our meteorologist team is aware of Hurricane Joaquin &amp; monitors weather conditions at all times. At this time a weather... 1/3,nealaa,3,1/3,1/3,Hi. Our meteorologist team is aware of Hurricane Joaquin & monitors weather conditions at all times. At this time a weather,21,0.33,7.00,JH
2,Thu,Oct,1,2015,@marxypoo ...the system &amp; when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies... 2/3,marxypoo,3,2/3,1/3,the system & when you are ready to board the aircraft the correct seat assignment will be given to you. My apologies,22,0.33,7.33,JH
3,Thu,Oct,1,2015,"@marxypoo Hi Marx. This is a known issue &amp; our IT team members are working on a fix. Be assured, once you make the change it is in... 1/3",marxypoo,3,1/3,1/3,"Hi Marx. This is a known issue & our IT team members are working on a fix. Be assured, once you make the change it is in",27,0.33,9.00,JH
4,Thu,Oct,1,2015,@bar_right Hi Pete. That is not good to hear. For questions or comments regarding the wifi service pls tweet @Gogo. A team member... 1/2,bar_right,2,1/2,1/2,Hi Pete. That is not good to hear. For questions or comments regarding the wifi service pls tweet . A team member,22,0.50,11.00,JH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,Tue,Oct,6,2015,@kate_wiley1 Welcome aboard! Enjoy! ML,kate_wiley1,1,NaN,1,Welcome aboard! Enjoy!,3,1.00,3.00,ML
531,Tue,Oct,6,2015,@MarneyReid I love it! Keep the momentum going! :-) AN,MarneyReid,1,NaN,1,I love it! Keep the momentum going! :-),8,1.00,8.00,AN
533,Tue,Oct,6,2015,"@jperrin83 Oh, no! Let me see if I can help. Pls follow and DM the file reference number. TP",jperrin83,2,NaN,1,"Oh, no! Let me see if I can help. Pls follow and DM the file reference number.",17,1.00,17.00,TP
534,Tue,Oct,6,2015,"@SA_Thompson Goooood morning Steven, welcome home, Sir! We are happy that you're flying with us again! Thanks for your loyalty to Delta! CK",SA_Thompson,1,NaN,1,"Goooood morning Steven, welcome home, Sir! We are happy that you're flying with us again! Thanks for your loyalty to Delta!",21,1.00,21.00,CK


In [ ]:
groupped_date = df_3.groupby(["date", "weekday"])["worker_id"].nunique().reset_index(name="count_workers").sort_values(by=["count_workers"], ascending=[False])
groupped_date

,date,weekday,count_workers
5,6,Tue,20
7,8,Thu,19
6,7,Wed,16
14,15,Thu,16
12,13,Tue,15
...,...,...,...
13,14,Wed,12
2,3,Sat,11
3,4,Sun,11
4,5,Mon,11


In [ ]:
groupped_weekday = groupped_date.groupby(["weekday"])["count_workers"].mean().reset_index(name="average_amount_workers").sort_values(by="average_amount_workers", ascending=False)
groupped_weekday

,weekday,average_amount_workers
5,Tue,17.50
4,Thu,16.33
6,Wed,14.00
1,Mon,12.50
3,Sun,12.50
0,Fri,12.00
2,Sat,11.50


##**QUESTION 4 - How many social replies are reasonable for a customer service representative to handle?**

In [ ]:
groupped_wk_date = df_3.groupby(["date", "worker_id"])["weight_float"].sum().reset_index(name="sum_weights").sort_values(by=["date", "sum_weights"], ascending=[True, False])
groupped_wk_date

,date,worker_id,sum_weights
8,1,NG,10.00
1,1,AB,9.00
5,1,KC,8.00
0,1,AA,5.00
3,1,DD,5.00
...,...,...,...
208,15,RD,3.00
193,15,AA,2.00
205,15,MD,2.00
196,15,BB,1.00


In [ ]:
groupped_w = groupped_wk_date.groupby(["worker_id"]).size().reset_index(name="days_active").sort_values(by="days_active", ascending = False)
groupped_w

,worker_id,days_active
24,RD,11
17,MD,10
12,JH,9
21,PL,9
8,DD,9
...,...,...
29,TP,3
30,VI,3
22,QB,3
7,CS,2


In [ ]:
average_daily_rep = groupped_wk_date.groupby(["worker_id"])["sum_weights"].mean().reset_index(name="average_daily_replies").sort_values(by="average_daily_replies", ascending = False)

av_daily_rep_fin = pd.merge(average_daily_rep, groupped_w, how="left", on = "worker_id")

av_daily_rep_fin

,worker_id,average_daily_replies,days_active
0,NG,13.71,7
1,PL,11.56,9
2,AA,9.25,8
3,ML,8.00,7
4,RS,8.00,8
...,...,...,...
28,HW,2.00,7
29,CK,1.60,5
30,QB,1.33,3
31,TH,1.00,1


In [ ]:
f = (av_daily_rep_fin["days_active"] >= 5)

filtered_replies = av_daily_rep_fin[f]
filtered_replies

,worker_id,average_daily_replies,days_active
0,NG,13.71,7
1,PL,11.56,9
2,AA,9.25,8
3,ML,8.00,7
4,RS,8.00,8
...,...,...,...
20,KM,3.38,8
22,SD,3.25,8
23,BB,3.00,8
28,HW,2.00,7


In [ ]:
column_statistics = filtered_replies['average_daily_replies'].describe()
column_statistics

count   24.00
mean     6.03
std      2.86
min      1.60
25%      4.38
50%      5.69
75%      7.33
max     13.71
Name: average_daily_replies, dtype: float64